# SocialTech Test task for Data Analyst position


## Task 1

In [ ]:
from datetime import date
import pandas as pd
import numpy as np
from scipy.stats import norm
from statsmodels.stats.proportion import proportions_ztest
import math

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/SocialTech_Test_task/raw_data.csv')

In [ ]:
df.describe()

,country_group,age_group,amount,successful_payment,split_group
count,58938.000000,58938.000000,10122.000000,10238.000000,58938.000000
mean,1.328617,2.989684,138.087253,0.665071,0.164291
std,0.741689,1.203772,83.758981,0.471989,0.370543
min,1.000000,1.000000,20.145000,0.000000,0.000000
25%,1.000000,2.000000,43.470000,0.000000,0.000000
50%,1.000000,3.000000,104.170000,1.000000,0.000000
75%,1.000000,4.000000,194.930000,1.000000,0.000000
max,4.000000,5.000000,335.340000,1.000000,1.000000


The earliest registered user in test group (split_group == 1)

In [ ]:
df[df['split_group'] == 1]['date_reg'].min()

'2021-07-23 14:17:49'

In [ ]:
df['date_reg'] = pd.to_datetime(df['date_reg'], format='%Y-%m-%d').dt.date

Let's see how many not-null values do we have in each group

In [ ]:
df.groupby(['split_group']).count()

,id_user,gender,date_reg,platform,id_traffic_source,country_group,age_group,system,date_payment,method,amount,successful_payment
split_group,,,,,,,,,,,,
0,49255,49255,49255,49255,49255,49255,49255,49255,8768,8768,8658,8768
1,9683,9683,9683,9683,9683,9683,9683,9683,1470,1470,1464,1470


In [ ]:
df[(df['successful_payment']==True) & pd.isna(df['amount'])]

,id_user,gender,date_reg,platform,id_traffic_source,country_group,age_group,system,date_payment,method,amount,successful_payment,split_group


Let's say we'd like to check if the total number of purchases has incresed in the test group (group with the new UI). For this purpose, let us use Normal approximation for Binomial distribution (z-test) for the following hypothoses set:

$ H_0: p_1 \leq p_0, $

$ H_1: p_1 > p_0 $

where $ p_1 $ -- conversion rate for test group (split_group == 1),

$ p_0 $ -- conversion rate for regular group (split_group == 0)


In [ ]:
df = df[df['date_reg'] >= date.fromisoformat('2021-07-24')]

Check how many users occured more than once in the dataset

In [ ]:
session_counts = df['id_user'].value_counts(ascending=False)
multi_users = session_counts[session_counts > 1].count()

print(f'There are {multi_users} users that appear multiple times in the dataset ~ {round(100 * multi_users / df.shape[0], 2)} % of the whole dataset')

There are 369 users that appear multiple times in the dataset ~ 1.9 % of the whole dataset


Since 369 is under 2% of the whole dataset, I assume we can just remove those users from the dataset

In [ ]:
# users_to_drop = session_counts[session_counts > 1].index

# df = df[~df['id_user'].isin(users_to_drop)]
# print(f'The updated dataset now has {df.shape[0]} entries')

In [ ]:
df_group_1 = df[(df['split_group'] == 1) & (df['date_reg'] >= date.fromisoformat('2021-07-24'))]
df_group_0 = df[(df['split_group'] == 0) & (df['date_reg'] >= date.fromisoformat('2021-07-24'))]

In [ ]:
n_elems_0 = df_group_0.shape[0]
n_elems_1 = df_group_1.shape[0]
print(f"Regular group size: {n_elems_0}, test group size: {n_elems_1}")

Regular group size: 9779, test group size: 9618


In [ ]:
n_purchases_0 = df_group_0[df_group_0['successful_payment'] == 1].shape[0]
n_purchases_1 = df_group_1[df_group_1['successful_payment'] == 1].shape[0]

In [ ]:
conversion_rate_0 = n_purchases_0 / n_elems_0
conversion_rate_1 = n_purchases_1 / n_elems_1
print(f"conversion rate in regular group is {conversion_rate_0:0.4f}, conversion rate in test group is {conversion_rate_1:0.4f}")

conversion rate in regular group is 0.0961, conversion rate in test group is 0.1032


In [ ]:
std_0 = np.sqrt(conversion_rate_0 * (1 - conversion_rate_0) / n_elems_0)
std_1 = np.sqrt(conversion_rate_1 * (1 - conversion_rate_1) / n_elems_1)
print(f"std_0 = {std_0:0.5f}, std_1 = {std_1:0.5f}")

std_0 = 0.00298, std_1 = 0.00310


In [ ]:
z_score = (conversion_rate_1 - conversion_rate_0) / np.sqrt(std_0**2 + std_1**2)
print(f"z-score is {z_score:0.2f}, with p-value {norm().sf(z_score):0.2f}")

z-score is 1.65, with p-value 0.05


Double-check with the proportions_ztest from statsmodels

In [ ]:
test_stat, p_value = proportions_ztest([n_purchases_0, n_purchases_1], [n_elems_0, n_elems_1], alternative='smaller')

In [ ]:
print(f"Computed z-test statistic: {test_stat:0.2f}, computed p-value: {p_value:0.2f}")

Computed z-test statistic: -1.66, computed p-value: 0.05


Since the computed p-value is greater than (or equal to) $ a = 0.05 $, we accept $ H_0 $ hypothesis on a confidence level 0.95, which means that the conversion rate in test group increased insignificantly

Used sources:
* https://cosmiccoding.com.au/tutorials/ab_tests
* https://towardsdatascience.com/ab-testing-with-python-e5964dd66143
* https://towardsdatascience.com/a-b-testing-with-binary-data-two-sample-proportions-z-test-5c222d7772ad

## Task 2

~~Results and calculations could be found here~~ *Here used to be link to the spreadsheet with data and calculations*

## Task 3.1

1. Количество регистраций новых пользователей по
дням по группам стран;
2. % писем, кликнутых в течение 10 минут после
отправки, по типам писем суммарно за последние 7
суток;

Таблицы:
1. `users` — таблица с пользователями сайта:
* `id` — уникальный идентификатор пользователя
* `email` — строковое поле с имейлом пользователя
* `id_country` — уникальный идентификатор страны
(соответствует `countries`.`id `)
* `date_reg` — дата и время регистрации пользователя
(например, «2017-02-27 10:58:05»)
2. `emails_sent` — таблица отправленных писем в рассылке:
* `id` — айди отправленного письма
* `id_user` — пользователь, которому отправили письмо
(соответствует `users`.`id`)
* `id_type` — тип письма
* `date_sent` — дата и время отправки письма
3. `emails_clicks` — таблица кликов по письмам:
* `id` — айди клика по письму
* `id_email` — айди письма, по которому был клик (соответствует
`emails_sent`.`id`)
* `date_click` — дата и время клика по письму
4. `countries` — список стран:
* `id` — идентификатор страны
* `name` — название страны в человекочитаемом виде
* `group` — к какой группе стран относится страна

### 1. Количество регистраций новых пользователей по дням по группам стран

```
select date(u.date_reg) date_reg,
	   c.country_group,
	   count(u.id) n_users
from users u
		join countries c on u.id_country = c.id
group by date(u.date_reg), c.country_group;
```

### 2. % писем, кликнутых в течение 10 минут после отправки, по типам писем суммарно за последние 7 суток; 

```
with last_week_emails as (
select es.id id_email,
        es.date_sent,
        es.id_type,
        ec.date_click,
      TIMESTAMPDIFF(MINUTE, es.date_sent, ec.date_click) AS min_delta
from emails_sent es
        left join emails_clicks ec on es.id = ec.id_email
where date(es.date_sent) >= NOW() - INTERVAL 7 DAY
),
clicked_in_ten as (
select id_type,
        count(id_email) n_clicked 
from last_week_emails
where min_delta <= 10
group by id_type
),
total_sent as (
select id_type, 
        count(id_email) n_sent 
from last_week_emails
group by id_type
)
select ts.id_type,
        100*n_clicked / n_sent
from total_sent ts
        left join clicked_in_ten ct on ts.id_type = ct.id_type;
```

Done and tested with MySQL 8.0 [here](https://paiza.io/projects/bvZPSxWie7nQ8twX6My-kQ?language=mysql)


```
-- drop table users;
-- drop table emails_sent;
-- drop table emails_clicks;
-- drop table countries;

create table users (id integer, email varchar(100), id_country integer, date_reg datetime);
insert into users (id, email, id_country, date_reg) values (1, 'albert@gmail.com', 1, '2017-02-27 10:58:05');
insert into users (id, email, id_country, date_reg) values (2, 'albert@gmail.com', 2, '2017-03-03 10:58:05');
insert into users (id, email, id_country, date_reg) values (3, 'albert@gmail.com', 1, '2017-03-03 10:58:05');
insert into users (id, email, id_country, date_reg) values (4, 'albert@gmail.com', 2, '2017-03-03 10:58:05');
insert into users (id, email, id_country, date_reg) values (5, 'albert@gmail.com', 3, '2017-03-07 10:58:05');
select * from users;

create table emails_sent (id integer, id_user integer, id_type varchar(100), date_sent datetime);
insert into emails_sent (id, id_user, id_type, date_sent) values (1, 1, 'promo', '2022-10-29 10:58:05');
insert into emails_sent (id, id_user, id_type, date_sent) values (2, 2, 'promo', '2022-10-30 10:58:05');
insert into emails_sent (id, id_user, id_type, date_sent) values (3, 3, 'security', '2022-10-31 10:58:05');
insert into emails_sent (id, id_user, id_type, date_sent) values (4, 4, 'security', '2022-10-31 10:58:05');
insert into emails_sent (id, id_user, id_type, date_sent) values (5, 5, 'digest', '2022-10-01 10:58:05');

create table emails_clicks (id integer, id_email integer, date_click datetime);
insert into emails_clicks (id, id_email, date_click) values (1, 1, '2022-10-29 11:22:05');
insert into emails_clicks (id, id_email, date_click) values (2, 2, '2022-10-30 10:59:05');
insert into emails_clicks (id, id_email, date_click) values (3, 3, '2022-10-31 10:59:05');
insert into emails_clicks (id, id_email, date_click) values (4, 4, '2022-10-31 11:01:05');
insert into emails_clicks (id, id_email, date_click) values (5, 5, '2022-10-02 10:58:05');

create table countries (id integer, name varchar(100), country_group varchar(100));
insert into countries (id, name, country_group) values (1, 'Ukraine', 'EMEA');
insert into countries (id, name, country_group) values (2, 'Poland', 'EU');
insert into countries (id, name, country_group) values (3, 'US', 'North America');
```






## Task 3.2

Дана таблица users_sessions, в которой находятся данные по
сессиям пребывания пользователей в онлайне на продукте:
* `id` - уникальный идентификатор сессии
* `id_user` - уникальный идентификатор пользователя
* `action` - поле, которое указывает на открытие или
закрытие сессии (два возможных значения: ‘open’ и ‘close’)
* `date_action` - таймстемп открытия/закрытия (в зависимости
от значения в поле ‘action’) сессии пребывания в онлайне

Нужно посчитать суммарное количество времени в
неокругленных часах, которое каждый из пользователей провел
на платформе в каждый из последних 10-ти дней (включая
текущий день).

```
WITH sessions AS (
SELECT DISTINCT us.id_user, DATE(us.date_action) date,
      CASE WHEN us.action = 'open' THEN us.date_action
          WHEN us.action = 'close' THEN LAG(us.date_action,1) OVER
          (PARTITION BY us.id_user ORDER BY us.date_action)
      END AS session_opened,
      CASE WHEN us.action = 'close' THEN us.date_action
          WHEN us.action = 'open' THEN LEAD(us.date_action,1) OVER
          (PARTITION BY us.id_user ORDER BY us.date_action)
      END AS session_closed
FROM users_sessions us
WHERE us.date_action > us.date_action - 10
),
session_dur AS (
SELECT s.id_user,
      s.date,
      s.session_opened,
      s.session_closed,
      TIMESTAMPDIFF(SECOND, s.session_opened, s.session_closed) session_duration_sec
FROM sessions s
),
last_10_days AS (
SELECT CURDATE() AS date
   UNION SELECT CURDATE() - INTERVAL 1 DAY
   UNION SELECT CURDATE() - INTERVAL 2 DAY
   UNION SELECT CURDATE() - INTERVAL 3 DAY
   UNION SELECT CURDATE() - INTERVAL 4 DAY
   UNION SELECT CURDATE() - INTERVAL 5 DAY
   UNION SELECT CURDATE() - INTERVAL 6 DAY
   UNION SELECT CURDATE() - INTERVAL 7 DAY
   UNION SELECT CURDATE() - INTERVAL 8 DAY
   UNION SELECT CURDATE() - INTERVAL 9 DAY
   UNION SELECT CURDATE() - INTERVAL 10 DAY
),
user_date AS (
SELECT u.id id_user,
       l10d.date
FROM last_10_days l10d, users u
)
SELECT ud.id_user,
       ud.date,
       SUM(sd.session_duration_sec) / 3600 total_dur_hours,
       sd.session_opened,
       sd.session_closed
FROM user_date ud
      LEFT JOIN session_dur sd ON ud.id_user = sd.id_user AND sd.date = ud.date
GROUP BY ud.id_user, ud.date, sd.session_opened, sd.session_closed
ORDER BY ud.date DESC, ud.id_user;    
```

```
create table users_sessions (id integer, id_user integer, action varchar(100), date_action datetime);
insert into users_sessions (id, id_user, action, date_action) values (1, 1, 'open', '2023-05-15 10:58:05');
insert into users_sessions (id, id_user, action, date_action) values (2, 1, 'open', '2023-05-17 13:27:05');
insert into users_sessions (id, id_user, action, date_action) values (3, 1, 'close', '2023-05-15 12:22:05');
insert into users_sessions (id, id_user, action, date_action) values (4, 1, 'close', '2023-05-17 13:29:05');
insert into users_sessions (id, id_user, action, date_action) values (5, 2, 'open', '2023-05-15 09:15:05');
insert into users_sessions (id, id_user, action, date_action) values (6, 2, 'close', '2023-05-15 09:25:05');
insert into users_sessions (id, id_user, action, date_action) values (7, 2, 'open', '2023-05-18 11:25:05');
insert into users_sessions (id, id_user, action, date_action) values (8, 2, 'close', '2023-05-18 12:55:05');
```